<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/BART_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# fairseqインストール(gitから)
!git clone https://github.com/utanaka2000/fairseq.git
%cd fairseq
!git fetch origin
!git checkout japanese_bart_pretrained_model
!git branch

In [ ]:
# インストール
! pip install .

In [ ]:
# その他必要なライブラリをインストール
!pip install zenhan sentencepiece tensorboard

In [ ]:
# fairseq有効化
!echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] = "/env/python"
os.environ['PYTHONPATH'] += ":/content/fairseq/"

!echo $PYTHONPATH

/env/python
/env/python:/content/fairseq/


In [ ]:
# インストールの確認
!pip show fairseq

In [ ]:
%cd /content
!wget http://lotus.kuee.kyoto-u.ac.jp/nl-resource/JapaneseBARTPretrainedModel/japanese_bart_base_1.1.tar.gz
!tar -zxvf japanese_bart_base_1.1.tar.gz

In [ ]:
! ls japanese_bart_base_1.1

bart_model.pt  dict.txt  sp.model


####  Juman++

In [ ]:
# jumanpp-2.0.0-rc3 download
%cd /content
!wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc3/jumanpp-2.0.0-rc3.tar.xz

In [ ]:
# unzip a file
!tar xvf jumanpp-2.0.0-rc3.tar.xz

In [ ]:
## jumanpp-2.0.0-rc3/src/core/analysis/analyzer.h
## 16,17行目
#  size_t pageSize = 48 * 1024 * 1024;
#  size_t maxInputBytes = 48 * 1024;

%%bash
sed -i -e "16c size_t pageSize = 48 * 1024 * 1024;" jumanpp-2.0.0-rc3/src/core/analysis/analyzer.h
sed -i -e "17c size_t maxInputBytes = 48 * 1024;" jumanpp-2.0.0-rc3/src/core/analysis/analyzer.h

! head -17 jumanpp-2.0.0-rc3/src/core/analysis/analyzer.h | tail -2

In [ ]:
## jumanpp-2.0.0-rc3/src/core/input/stream_reader.h
## 27,28行目
#  u64 maxInputLength_ = 48 * 1024;
#  u64 maxCommentLength_ = 48 * 1024;

%%bash
sed -i -e "27c u64 maxInputLength_ = 48 * 1024;" jumanpp-2.0.0-rc3/src/core/input/stream_reader.h
sed -i -e "28c u64 maxCommentLength_ = 48 * 1024;" jumanpp-2.0.0-rc3/src/core/input/stream_reader.h

! head -28 jumanpp-2.0.0-rc3/src/core/input/stream_reader.h | tail -2

In [ ]:
# build jumanpp
%cd jumanpp-2.0.0-rc3/
!mkdir build
!cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
!make

In [ ]:
# install jumanpp
!sudo make install

In [ ]:
! pwd
! rm /content/jumanpp-2.0.0-rc3.tar.xz

In [ ]:
! ls -alh /content